In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
# import input_data
import tensor_mnist.mnist as input_data

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets('data/', one_hot=True)

filepath:data/train-images-idx3-ubyte.gz
Extracting data/train-images-idx3-ubyte.gz
filepath:data/train-labels-idx1-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
filepath:data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
filepath:data/t10k-labels-idx1-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


参数初始化

In [3]:
# NETWORK TOPOLOGIES
n_hidden_1 = 256 
n_hidden_2 = 128 
n_input    = 784  # 28*28
n_classes  = 10  

# INPUTS AND OUTPUTS 占位
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
    
# NETWORK PARAMETERS
stddev = 0.1

# x=n*784,  y=n*10
# 第一层256个神经元，第二层128个神经元
# w1=784*256,w2=256*128,b1=256,b2=128
# 最后一层为输出值128*10
# w和b都进行高斯初始化
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=stddev))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
print ("NETWORK READY")

NETWORK READY


定义前向传播

In [4]:
# 定义前向传播，得到10个类别的得分值，最终输出层不用加sigmoid
def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, _weights['w1']), _biases['b1'])) 
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, _weights['w2']), _biases['b2']))
    # 最终数据结果是得分，所以可不用加sigmoid,结果的shape是n*10
    #return(tf.nn.softmax(tf.matmul(layer_2, _weights['out']) + _biases['out']))
    return (tf.matmul(layer_2, _weights['out']) + _biases['out'])

定义反向求导：  
1、损失函数  
2、梯度下降最小化损失函数  
3、计算准确率

In [5]:
# 预测值
pred = multilayer_perceptron(x, weights, biases)

# 用交叉熵损失函数，传进去预测值和真实值
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y)) 
# 梯度下降
optm = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cost) 
# 算准确率
corr = tf.equal(tf.argmax(pred, axis=1), tf.argmax(y, axis=1))    
accr = tf.reduce_mean(tf.cast(corr, "float"))

# 初始化
init = tf.global_variables_initializer()
print ("FUNCTIONS READY")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

FUNCTIONS READY


In [6]:
training_epochs = 10
batch_size      = 100
display_step    = 5
# LAUNCH THE GRAPH
# 加载计算图
sess = tf.Session()
# 运行初始化
sess.run(init)
# OPTIMIZE
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # ITERATION
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y: batch_ys}
        # 运行梯度下降操作
        sess.run(optm, feed_dict=feeds)
        #  运行求损失值操作
        avg_cost += sess.run(cost, feed_dict=feeds)
    avg_cost = avg_cost / total_batch
    # DISPLAY
    if (epoch+1) % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys}
        
#         predict = sess.run(pred,feed_dict=feeds)
#         #print(predict)
#         print(sess.run(tf.argmax(predict, axis=1)))
        
        train_acc = sess.run(accr, feed_dict=feeds)
        print ("TRAIN ACCURACY: %.3f" % (train_acc))
        feeds = {x: mnist.test.images, y: mnist.test.labels}
        test_acc = sess.run(accr, feed_dict=feeds)
        print ("TEST ACCURACY: %.3f" % (test_acc))
print ("OPTIMIZATION FINISHED")

Epoch: 004/010 cost: 2.260462202
TRAIN ACCURACY: 0.240
TEST ACCURACY: 0.231
Epoch: 009/010 cost: 2.215562756
TRAIN ACCURACY: 0.400
TEST ACCURACY: 0.367
OPTIMIZATION FINISHED
